In [ ]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [ ]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return (data.get_exam_duration(course)//15)

    return int(exam_length(course)//15)
#Dhruv's part:

# session_spans = {s: begin_end_pairs(session_length(s)) for s in range(n_sessions)}
# print(data.begin_end_pairs(exam_slots(1)))
exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [ ]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [ ]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course

In [ ]:


# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [ ]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for c in range(data.n_courses)
#                           for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
#               for d in range(data.n_days)
#               for r in range(data.n_rooms)
#               for t in range(data.n_times)),
#               name = "room_capacity_constraint"
#              )

P = tt.addVars(
    [
        (c, r, b, e, d)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.INTEGER,
    name = 'P',
    lb = 0, 
    ub = 230,
)


for c,r,b,e,d in P:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(P[c,r,b,e,d] >=0, name = 'P1_constraint')
    tt.addConstr(P[c,r,b,e,d] <= room_capacity*E[c,b,e,d], name = 'P2_constraint')
    tt.addConstr(P[c,r,b,e,d] <= C[c,r]*(1-E[c,b,e,d]), name = 'P3_constraint')
    tt.addConstr(P[c,r,b,e,d] >= (C[c,r] - room_capacity)*(1-E[c,b,e,d]), name = 'P4_constraint')


tt.update()

tt.addConstrs(
    (gp.quicksum(P[c, r, b, e, d] 
                 for r in range(data.n_rooms) 
                 for b, e in covering_sessions(c, t)) <= data.get_room_capacity(r)
     for d in range(data.n_days) 
     for c in range(data.n_courses) 
     for t in range(data.n_times)),
    name='room_capacity_constraint'
)

tt.update()


In [ ]:
# Constraint 3: All students for a course should write the exam 

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for r in range(data.n_rooms)
#                           for b,e in covering_sessions(c, t)) <= data.get_course_strength(c)
#               for d in range(data.n_days)
#               for c in range(data.n_courses)
#               for t in range(data.n_times)),
#               name = 'course_enrollment_constraint'
#              )

# # Constraint 3.1: All students for a course should write the exam
# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                            for d in range(data.n_days)
#                           for r in range(data.n_rooms)
#                           for t in range(data.n_times)
#                           for b,e in covering_sessions(c, t)) == data.get_course_strength(c)
#               for c in range(data.n_courses)),
#               name = 'course_enrollment_constraint'
#              )

# Constraint 3.1: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)



# tt.addConstrs(
#     (gp.quicksum(P[c, r, b, e, d] 
#                  for r in range(data.n_rooms) 
#                  for b, e in covering_sessions(c, t)) == data.get_course_strength(c)
#      for d in range(data.n_days) 
#      for c in range(data.n_courses) 
#      for t in range(data.n_times)),
#     name='course_enrollment_constraint'
# )

tt.update()


In [ ]:
# # Constraint 4: Student clash constraint

# # for each student, for each day, for each time slot, at most one exam
# # sigma Ecbed <= 1 for courses enrolled by student s


tt.addConstrs((gp.quicksum(E[c,b,e,d] for c in data.get_courses_student(s) 
                for b,e in covering_sessions(c, t)) <= 1
                for s in range(data.n_students)
                for d in range(data.n_days)
                for t in range(data.n_times)),
              name = 'student_clash_constraint'
             )

# tt.update()

In [ ]:
# Constraint 5: No course can have more than 3 rooms alloted


M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)



for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

# tt.addConstrs(
#     (
#         # gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
#         gp.quicksum(min(1, C[c, r]) for r in range(data.n_rooms)) <= 3
#         for c in range(data.n_courses)
#     ),
#     name="course_room_constraint",
# )

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [ ]:
# Constraint 6: Lab exams are conducted in labs
# check_cse_lab_room and check_ece_lab_room 

tt.addConstrs(
    ()
)

## Soft Constraints

In [ ]:
# 1. No 2 exams on a day for a student

# S1 = gp.quicksum(
#     (gp.quicksum(E[c,b,e,d] for c in (data.get_courses_student(s))
#     for b,e in exam_spans[c])) >=2
#     for d in range(data.n_days)
#     for s in range(data.n_students)
# )
 # count the number of >=2 for each student, each day and return it into S1

S1 = tt.addVars(
    [
        (s, d)
        for s in range(data.n_students)
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name = 'S1',
)


for s,d in S1:
    tt.addConstr(
        gp.quicksum(E[c,b,e,d] for c in (data.get_courses_student(s))
        for b,e in exam_spans[c]) +  S1[s, d]*100 >=2
    )
    

tt.update()


## Objective Function

In [ ]:
objective = S1.sum()
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [ ]:
# Maximize Gaps between exams for a student / batch
# tt.optimize()

# def early_stop_callback(model, where):
#     if where == GRB.Callback.MIP:
#         runtime = model.cbGet(GRB.Callback.RUNTIME)
#         mipgap = model.cbGet(GRB.Callback.MIPGAP)
#         print("runtime:", runtime)
#         print("mipgap:", mipgap)
#         if runtime > 120 and mipgap > 0.15:
#             print("Early stopping triggered: runtime > 120 sec and MIPGap > 15%")
#             model.terminate()

# # Optimize with the early stopping callback
# tt.optimize(early_stop_callback)

import time


def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 20s
    if time.time() - model._time > 60:
        model.terminate()

tt._cur_obj = float('inf')
tt._time = time.time()

tt.optimize(callback=cb)

In [ ]:
E_sol = tt.getAttr('x', E)
C_sol = tt.getAttr('x', C)
M_sol = tt.getAttr('x', M)
D1_sol = tt.getAttr('x', D1)
D2_sol = tt.getAttr('x', D2)

#Get all the values from E_sol whose value is 1
solution=[]
for i in E_sol:
    if E_sol[i] == 1:
        solution.append(i)

# for i in solution:
#     print(i)

In [ ]:
# print(C_sol)
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

# for i in solution_rooms:
#     print(i)

In [ ]:
for c,b,e,d in solution:
    l = []
    for(r) in range(data.n_rooms):
        if C_sol[c,r] > 0:
            l.append((data.get_room_num(r), C_sol[c,r]))
    print(f"Course {data.course_id_code[c]} has {data.get_course_strength(c)} and is scheduled in slot {data.times[b]} to {data.times[e]} on day {data.days[d]} in room {l}")

In [ ]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")